In [1]:
 !pip install rioxarray "xarray[complete]" geopandas


  Obtaining dependency information for rioxarray from https://files.pythonhosted.org/packages/4e/34/8ee402df8a49228b6aae89ab2127cdd5aa0ba0976904af7d55270f4f30b8/rioxarray-0.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for xarray[complete] from https://files.pythonhosted.org/packages/cc/21/c3a9d3017c131cea3c2967957b05448ffc81cafced239796bbbdc98073e4/xarray-2023.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/cb/7b/2fb6ab376c78d2e60812ad0bf4b24889309d2aeb543f54efcd586c69907f/geopandas-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for rasterio>=1.2 from https://files.pythonhosted.org/packages/c8/7e/08d80abba94a41a6102b432897fe21e99896120fad2766b4875bfc06a8d2/rasterio-1.3.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pyproj>=2.2 from https://files.pythonhosted.org/packages/24/18/0a0895fd8c1136fea0aed5e0ee290e59

In [2]:
import rioxarray as rx
import pandas as pd
import boto3
import xarray as xr
import glob
import shutil
import os
import zarr
import s3fs
import geopandas as gpd
from dask.diagnostics import ProgressBar

In [3]:
pathS3 = "s3://climate-action-datalake/zone=raw/source=agera5"
localPath = 'data/*.nc'

In [4]:
def convertRadiation(k):
    c = k/1000000
    return c

In [5]:
allFiles = xr.open_mfdataset(localPath,parallel=True)

In [6]:
print('Hola mundo')

Hola mundo


In [7]:
allFiles = allFiles.apply(convertRadiation)

In [19]:
compressor = zarr.Blosc(cname='zstd')
encoding = {vname: {'compressor': compressor,'chunks': (1,1801,3600)} for vname in allFiles.data_vars}


In [20]:
s3 = s3fs.S3FileSystem()
store = s3fs.S3Map(root="s3://climate-action-datalake/zone=raw/source=agera5/variable=solarRadiationFlux.zarr", s3=s3, check=False)

In [21]:
with ProgressBar():
    result = allFiles.to_zarr(store=store, encoding=encoding, consolidated=True)

[########################################] | 100% Completed | 50m 8ss


In [22]:
print("He finalizado")

He finalizado
